In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from gensim.corpora import WikiCorpus
import logging
import collections
import math
import os
import six
import random
import zipfile
import sys
import pandas as pd
import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

/Users/sainishanthdilly/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def read_data(filename):
    with open(filename) as f:
        lines = f.read().split(' ')
    return lines

In [4]:
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [5]:
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        if data_index == len(data):
            buffer[:] = data[:span]
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [6]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

    plt.savefig(filename)

In [10]:
url = 'http://mattmahoney.net/dc/'
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)

In [11]:
outp = "simplewiki-20180220-pages-articles-multistream_output.txt"
inp =  "simplewiki-20180220-pages-articles-multistream.xml.bz2"
output = open(outp, 'w')
wiki = WikiCorpus(inp, lemmatize=False, dictionary={})

<_io.TextIOWrapper name='simplewiki-20180220-pages-articles-multistream_output.txt' mode='w' encoding='UTF-8'>


In [12]:
i = 0
for text in wiki.get_texts():
    output.write(" ".join(text) + "\n")
    i = i + 1
    if (i % 1000 == 0):
        logger.info("Saved " + str(i) + " articles")
output.close()
logger.info("Finished Saved " + str(i) + " articles")

In [13]:
vocabulary = read_data(outp)
print('Data size', len(vocabulary))

Data size 23344484


In [14]:
vocabulary_size = 50000
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)

In [15]:
del vocabulary  # Hint to reduce memory.

In [16]:
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 1187634], ('the', 1494190), ('of', 772327), ('in', 599958), ('and', 576489)]
Sample data [115, 6, 1, 62, 832, 2, 1, 78, 4, 601] ['april', 'is', 'the', 'th', 'month', 'of', 'the', 'year', 'and', 'comes']


In [17]:
# sliding pointer over all the data
data_index = 0

In [19]:
for i in range(100): # print first 100 words
    print(reverse_dictionary[data[i]],end=' ')
batch, labels = generate_batch(batch_size=16, num_skips=4, skip_window=2)
cur_target = ''
cur_context = []
for i in range(16):
    if cur_target!=batch[i]:
        if cur_target!='':
            print(reverse_dictionary[cur_target], cur_context)
        cur_target = batch[i]
        cur_context = [reverse_dictionary[labels[i, 0]]]
    else:
        cur_context.append(reverse_dictionary[labels[i, 0]])
        
print(reverse_dictionary[cur_target], cur_context)

april is the th month of the year and comes between march and may it is one of four months to have days april always begins on the same day of week as july and additionally january in leap years april always ends on the same day of the week as december april flowers are the sweet pea and daisy its birthstone is the diamond the meaning of the diamond is innocence the month spring flowers in april in the northern hemisphere april comes between march and may making it the fourth month of the year it also comes first the ['and', 'month', 'of', 'year']
year ['comes', 'of', 'the', 'and']
and ['the', 'between', 'comes', 'year']
comes ['and', 'between', 'march', 'year']


In [20]:
# reset sliding pointer over all the data
data_index = 0

In [21]:
batch_size = 128
embedding_size = 60  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

In [22]:
# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
np.random.seed(10)
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
# add some known words
known_words = np.array([dictionary['april'],dictionary['year'],dictionary['world'],dictionary['country']])
#know_word = np.array([])
valid_examples = np.append(known_words, valid_examples)

In [24]:
num_sampled = 64    # Number of negative examples to sample.

In [25]:
graph = tf.Graph()

In [27]:
with graph.as_default():
  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)
  print(similarity)
  # Add variable initializer.
  init = tf.global_variables_initializer()

Tensor("MatMul_3:0", shape=(20, 50000), dtype=float32)


In [28]:
num_steps = 20001

In [29]:
with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  314.9900817871094
Nearest to april: prinze, kimura, aggressively, nizao, profit, haldeman, applauded, cinderella,
Nearest to year: sham, airships, carf, nawab, flags, contempt, lans, ledbetter,
Nearest to world: uda, surgeons, schindler, grades, paperback, spool, axle, schwartz,
Nearest to country: swain, extant, tamura, flop, evidences, pains, sel, transitioned,
Nearest to his: hymns, doubleday, towne, measurements, note, mauser, bora, shafts,
Nearest to with: neil, kurek, iserlohn, puckett, ussel, pacifica, supervises, harpist,
Nearest to there: tennis, grandfathers, telephones, same, endymion, indycar, löw, want,
Nearest to new: satire, remainder, cerri, mena, monestier, shoreline, applying, mutated,
Nearest to states: corps, madhavan, bossa, icarus, breastfeeding, otello, sacs, passed,
Nearest to in: invisible, sergey, warped, monitoring, urbain, mrt, aidan, compatibility,
Nearest to part: domain, federated, anchovies, suffocation, heated, adr

In [31]:
try:
  # pylint: disable=g-import-not-at-top
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels)

except ImportError:
    print('Please install sklearn, matplotlib, and scipy to show embeddings.')

In [33]:
datafr = pd.read_csv("Old.csv")

basic_word_lis = datafr.columns.values.tolist()

newLis = datafr.iloc[1:]
print(basic_word_lis)

dict1 = {}
for i in range(len(count)):
    if count[i][0] in basic_word_lis:
        dict1[count[i]] = i

list_final_embeddings_simple_Wiki = []
for key in dict1:
    list_final_embeddings_simple_Wiki.append(final_embeddings[dict1[key]])
print(list_final_embeddings_simple_Wiki)

a = tf.placeholder(tf.float32, shape=[None], name="vector_placeholder_simplewiki")
b = tf.placeholder(tf.float32, shape=[None], name="vector_placeholder_basic")
normalize_a = tf.nn.l2_normalize(a,0)        
normalize_b = tf.nn.l2_normalize(b,0)
cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
sess=tf.Session()

for key in dict1:
    cos_sim=sess.run(cos_similarity,feed_dict={a: datafr[key[0]],b: final_embeddings[dict1[key]] })
    print("Cosine Similiary of ", key[0], cos_sim)

sess.close()

['the', 'of', 'in']
[array([-0.28205192, -0.11378258,  0.27501428, -0.12458557, -0.21936645,
       -0.10316052, -0.11320788, -0.04106405,  0.0241688 ,  0.01947027,
        0.00786876, -0.05021696,  0.0916545 , -0.18406476,  0.08837993,
       -0.08763159,  0.18055297,  0.07370386,  0.05986102, -0.11189407,
        0.07325902, -0.02358193, -0.02638974,  0.2833917 , -0.11093546,
       -0.01607505,  0.06869192, -0.01602526, -0.08043829, -0.25877067,
        0.08388104,  0.00499224, -0.15483576, -0.12541945, -0.16265844,
       -0.10827539,  0.09608718, -0.27601278, -0.221907  ,  0.04579188,
        0.09454805,  0.07013384, -0.02252553,  0.04753075,  0.08033574,
        0.08738975,  0.03322274, -0.06390604,  0.07417992,  0.05982525,
        0.03346559,  0.04952756,  0.2215307 , -0.07427252, -0.04072355,
       -0.13680652,  0.11554171, -0.2491798 ,  0.04700441,  0.16385177],
      dtype=float32), array([-0.142543  , -0.03229989, -0.09574114, -0.13398823, -0.14520948,
       -0.00291673, 